In [2]:
# !pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 487.6 kB/s eta 0:01:20
   ---------------------------------------- 0.1/38.8 MB 558.5 kB/s eta 0:01:10
   ---------------------------------------- 0.1/38.8 MB 847.9 kB/s eta 0:00:46
   ---------------------------------------- 0.2/38.8 MB 953.7 kB/s eta 0:00:41
   ---------------------------------------- 0.3/38.8 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.5/38.8 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.6/38.8 MB 1.5 MB/s eta 0:00:26
    --------------------------------------- 0.7/38.8 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.7/38.8 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.8/38.8 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.9/38.8 MB 1.7 MB/s eta 0:00:22
   - 

In [5]:
import cv2
import numpy as np

# Đường dẫn tới các tệp cần thiết
weights_path = 'Assets/Trained_Model/yolov3/yolov3.weights'
config_path = 'Assets/Trained_Model/yolov3/yolov3.cfg'
names_path = 'Assets/Trained_Model/coco.names'

# Đọc các nhãn từ tệp coco.names
with open(names_path, 'r') as f:
    classes = f.read().strip().split('\n')

# Tải mô hình YOLO
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# Hàm để nhận diện các đối tượng trong khung hình
def detect_objects(frame):
    (H, W) = frame.shape[:2]
    
    # Tạo blob từ khung hình đầu vào và thực hiện truyền tiến qua mạng YOLO
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    unconnected_layers = net.getUnconnectedOutLayers()
    
    print(f"Layer names: {layer_names}")
    print(f"Unconnected layers: {unconnected_layers}")
    
    output_layers = [layer_names[i - 1] for i in unconnected_layers.flatten()]
    print(f"Output layers: {output_layers}")
    
    outputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []
    
    # Duyệt qua các đầu ra từ các lớp đầu ra YOLO
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Áp dụng non-maxima suppression để loại bỏ các bounding boxes yếu
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in np.random.randint(0, 255, size=(3,))]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[class_ids[i]], confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return frame

# Đường dẫn tới video
video_path = 'Assets/Traffic_Data/Traffic_Violations/No_Violation_Video/No Violation 1 - Made with Clipchamp.mp4'

# Mở video
cap = cv2.VideoCapture(video_path)

# Kiểm tra nếu video mở thành công
if not cap.isOpened():
    print("Error: Không thể mở video.")
    exit()

# Đọc từng khung hình từ video và nhận diện
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Nhận diện đối tượng
    frame = detect_objects(frame)
    
    # Hiển thị khung hình kết quả
    cv2.imshow('Video', frame)
    
    # Thoát khi nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng bộ nhớ và đóng các cửa sổ
cap.release()
cv2.destroyAllWindows()



Layer names: ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn